In [21]:
import requests
from bs4 import BeautifulSoup
import ollama
import textwrap
import gradio as gr


In [22]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        self.links = [link.get('href') for link in soup.find_all('a') if link.get('href')]

In [23]:
class WebsiteChatbot:
    def __init__(self, website):
        self.website = website

    def ask(self, question):
        prompt = f"""
        You are an AI assistant with knowledge about the following website: {self.website.url}.
        Website Title: {self.website.title}
        Extracted Content:
        {self.website.text[:3000]}  # Limiting to 3000 chars for context
        
        User Question: {question}
        Answer in well-formatted Markdown style:
        """
        response = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": prompt}])
        formatted_response = textwrap.fill(response["message"]["content"], width=180)
        return formatted_response

In [24]:
def chat_with_website(url, question):
    if not url.strip():
        return "Please enter a valid website URL."
    if not question.strip():
        return "Please enter a question."
    
    site = Website(url)
    chatbot = WebsiteChatbot(site)
    return chatbot.ask(question)

In [25]:
demo = gr.Interface(
    fn=chat_with_website,
    inputs=[gr.Textbox(label="Website URL"), gr.Textbox(label="Your Question")],
    outputs=gr.Markdown(label="AI Response"),
    title="Website Chatbot with LLaMA 3.2",
    description="Enter a website URL and ask a question about its content."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


this is being done with llama3.2 which is agood model, its open source and so you can run it locally at no cost. However, we would get much better results if we used gpt-4o-mini because it is more powerful but then you need to pay for the API - nevertheless this is a useful tool to save you a lot of time. 